In [8]:
from sodapy import Socrata
from plotly import graph_objects as go
from datetime import datetime, date, time, timedelta
from datetime import datetime
# from highcharts import Highchart
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import plotly.express as px
import datetime
import numpy as np
import utils
import plotly
%matplotlib inline

from matplotlib.dates import DateFormatter
pd.options.display.max_colwidth = 200
import warnings
warnings.filterwarnings('ignore')

### Get Data from Socrata

In [22]:
## Build client
usrname = "jjensen@mointern.nyc.gov"
pword = "Timspw19!"
client = Socrata("data.cityofnewyork.us","TnnbmTZvKbFz7tJ4NpzkfUG6C", username=usrname, password=pword)

## Make API calls
auth_json_downloads = client.get("beb8-u7wp", limit=2000000)
auth_json_dailyviews = client.get("qiif-mxcx", limit=50000)
auth_json_authentications = client.get("8aih-pmax", limit = 20000)
auth_json_datasetfacts = client.get("gzid-z3nh", limit = 500000)
auth_json_public_inventory = client.get("5tqd-u88y", limit=5000)
auth_json_api_calls = client.get("u7pi-wq4y", limit=2000)
auth_json_the_plan = client.get("qj2z-ibhs", limit=5000)


## Get the private asset inventory
## only necessary because the columns `asset_type` is only in the private and not the public asset inventory
auth_json_asset_inventory = client.get("r8cp-r4rc", limit = 40000)

### Generate base dataframes to build off of throughout the notebook

In [4]:
## Convert json files into pandas dataframes
dv = pd.DataFrame(auth_json_dailyviews)
dauth = pd.DataFrame(auth_json_authentications)
dfacts = pd.DataFrame(auth_json_datasetfacts)
ddownloads = pd.DataFrame(auth_json_downloads)
public = pd.DataFrame(auth_json_public_inventory)
private = pd.DataFrame(auth_json_asset_inventory)
the_plan = pd.DataFrame(auth_json_the_plan)

### Data Cleaning

In [5]:
# Create merged_filter, the dataframe that has only assets defined as datasets
filter_list = ['dataset','filter', 'gis map']
## Cleaning dates entered in the wrong format
bad_index_list = list(public.query('date_made_public == "August 9, 2-019"').index)
for val in bad_index_list:
    public.at[val, 'date_made_public'] = '8/9/2019'

# need to merge in private so that we can get the asset type
dupes = private[['u_id', 'agency', 'type','name', 'parent_uid']]

public = utils.date_transformation(public, 'date_made_public')
upUntilfy20 = utils.apply_date_mask(public, 'date_made_public', fy=False)
merged_total = pd.merge(public, dupes, how='left', on='u_id')
merged_filter = merged_total[merged_total.type.isin(filter_list)]

## Number of Datasets on NYC Open Data Portal

In [6]:
published_datasets_by_fy = utils.create_fy_columns(merged_filter, 'date_made_public')
published_assets_by_fy = utils.create_fy_columns(merged_total, 'date_made_public')

In [17]:
num_datasets = pd.DataFrame(published_datasets_by_fy[['until_fy_16', 'until_fy_17', 'until_fy_18', 'until_fy_19', 'until_fy_20']].sum())\
    .reset_index().rename(columns={'index':'FY', 0:'Number of Datasets on ODP'})

num_assets = pd.DataFrame(published_assets_by_fy[['until_fy_16', 'until_fy_17', 'until_fy_18', 'until_fy_19', 'until_fy_20']].sum())\
    .reset_index().rename(columns={'index':'FY', 0:'Number of Assets on ODP'})

assets_by_type = pd.DataFrame(merged_total.groupby('type').size())\
    .reset_index().sort_values(ascending=False, by=0)\
    .rename(columns={'type':'asset_type', 0:'Count'})

In [14]:
fig = px.bar(num_datasets, x='FY', y='Number of Datasets on ODP')
# fig.update_layout(yaxis=num_datasets['Number of Datasets on ODP'])

fig.show()

In [21]:
# Number of Assets per FY

fig = px.bar(num_assets, x='FY', y='Number of Assets on ODP')

fig.show()

In [19]:
# Number of Assets by Asset Type

fig = px.bar(assets_by_type, x='asset_type', y='Count')

fig.show()

In [ ]:
# Number of Unique Agencies that Published Dataset per Fiscal Year

published_datasets_by_fy.groupby(['fy_16','fy_17','fy_18','fy_19'], as_index=True)\
    ['agency_x'].nunique().reset_index()